# Training YOLOv8 - Python API

For comparison with launching the training from terminal (`yolo task=detect [...]`)


## Training with AMP

AMP = "Automatic Mixed Precision"

In [1]:
%pip install -U ultralytics pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 kB 11.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.2.8
    Uninstalling ultralytics-8.2.8:
      Successfully uninstalled ultralytics-8.2.8
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import time
import sys
import ultralytics
from IPython import display
import torch

display.clear_output()

ultralytics.checks()

Ultralytics YOLOv8.2.9 🚀 Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA A40, 45416MiB)
Setup complete ✅ (4 CPUs, 7.7 GB RAM, 33.8/101.5 GB disk)


In [3]:
# Training parameters:
n_epochs = 200
n_patience = 80
batch_size = 4
amp_flag = True

HOME_tomato_leaves_segm = os.getcwd()
print("Home: ", HOME_tomato_leaves_segm)

torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False
torch.backends.cudnn.enabled = True

cudnn_flg = torch.backends.cudnn.is_available()
print("cuDNN available: ", cudnn_flg)
if cudnn_flg:
    print("cuDNN version: ", torch.backends.cudnn.version())

Home:  /home/jovyan/FREISA/code/training-yolo
cuDNN available:  True
cuDNN version:  8902


### Model selection

Available YOLOv8 models (from least number of parameters to highest):

* YOLOv8n -> Nano
* YOLOv8s -> Small
* YOLOv8m -> Medium
* YOLOv8l -> Large
* YOLOv8x -> Extra-large

Notice that with bigger models not only the training takes much more time, but also inference does - meaning that, especially on a low-power device as the OAK-D lite, deploying the bigger models may be problematic.

In [4]:
## Select model:
# yolo_model = 'yolov8n.pt'  # Use YOLOv8 'Nano'
# yolo_model = 'yolov8s.pt'  # Use YOLOv8 'Small'
yolo_model = 'yolov8m.pt'  # Use YOLOv8 'Medium'
# yolo_model = 'yolov8l.pt'  # Use YOLOv8 'Large'
# yolo_model = 'yolov8x.pt'  # Use YOLOv8 'Extra-large'

In [6]:
!unzip "Tomato Leaves Detection.v1i.yolov8.zip"

Archive:  Tomato Leaves Detection.v1i.yolov8.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/100_46341_jpg.rf.63aa23e5da862327a5ba86a056f1f9be.jpg  
 extracting: test/images/18_20201028_556416_jpg.rf.0a8c4c647344469162b574bcbd3a471e.jpg  
 extracting: test/images/18_20201028_557168_jpg.rf.134a8feafc3c84fd10714bc448fba73d.jpg  
 extracting: test/images/18_20201028_569179_jpg.rf.8cccbcbd02a85af0d0bb81a0d7535551.jpg  
 extracting: test/images/18_20201028_569215_jpg.rf.1dc27dbac8821595826fafe5e7ebefbe.jpg  
 extracting: test/images/18_20201028_569218_jpg.rf.a516e420dd9fc68bd6cb8fad1d89d422.jpg  
 extracting: test/images/18_20201028_569218_jpg.rf.f13a2f167a3a58d00bbfd8044604c335.jpg  
 extracting: test/images/18_20201028_569399_jpg.rf.7487585f1aec0bdb642a6ce431c33edd.jpg  
 extracting: test/images/18_20201118_1179132_jpg.rf.2c7d238e5d9e32d4dae0a86356fe8

In [7]:
model = ultralytics.YOLO(yolo_model)
t_start = time.time()
results = model.train(data=os.path.join(HOME_tomato_leaves_segm, 'data.yaml'), epochs=n_epochs, patience=n_patience, batch=batch_size, imgsz=640, amp=amp_flag)
t_stop = time.time()

Ultralytics YOLOv8.2.9 🚀 Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA A40, 45416MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/home/jovyan/FREISA/code/training-yolo/data.yaml, epochs=200, time=None, patience=80, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

100%|██████████| 755k/755k [00:00<00:00, 25.0MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

 21                  -1  2   4207104  ultralytics.nn.modules.block.C2f             [960, 576, 2]                 
 22        [15, 18, 21]  1   3776854  ultralytics.nn.modules.head.Detect           [2, [192, 384, 576]]          
Model summary: 295 layers, 25857478 parameters, 25857462 gradients, 79.1 GFLOPs

Transferred 469/475 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


100%|██████████| 6.23M/6.23M [00:00<00:00, 75.0MB/s]


...

    142/200      2.44G     0.7995     0.5735      1.112         12        640: 100%|██████████| 268/268 [00:15<00:00, 17.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:00<00:00, 21.81it/s]

                   all        105        608      0.633      0.554       0.57      0.321
EarlyStopping: Training stopped early as no improvement observed in last 80 epochs. Best results observed at epoch 62, best model saved as best.pt.
To update EarlyStopping(patience=80) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



142 epochs completed in 0.689 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 52.0MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.9 🚀 Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA A40, 45416MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:00<00:00, 15.21it/s]


                   all        105        608      0.564       0.62      0.604      0.356
          Healthy-Leaf        105        265      0.615      0.758      0.715      0.455
        Unhealthy-Leaf        105        343      0.513      0.481      0.493      0.256
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train2


In [ ]:
print(f"Training time, AMP enabled: {t_stop - t_start} seconds")

### Validation

In [ ]:
model.val()